In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/Applied \AI \course \Assignments/Case \Study \2

/content/drive/MyDrive/Applied AI course Assignments/Case Study 2


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import cv2
from skimage import io
from skimage import transform
from skimage import color
import pickle

import pickle
from tqdm.notebook import tqdm

from sklearn.metrics import mean_squared_error

import tensorflow as tf

## load the images and counts from folders

In [ ]:
# Load the images and counts
with open('Images.pkl', 'rb') as f:
  images = pickle.load(f)

with open('Counts.pkl', 'rb') as f:
  counts = pickle.load(f)


In [ ]:
A1_image_paths, A1_counts = [],[]
A2_image_paths, A2_counts = [],[]
A3_image_paths, A3_counts = [],[]
A4_image_paths, A4_counts = [],[]

for image, count in zip(images, counts):
    folder = image.split("/")[1]

    if folder == 'A1':
        A1_image_paths.append(image)
        A1_counts.append(count)
    elif folder == 'A2':
        A2_image_paths.append(image)
        A2_counts.append(count)
    elif folder == 'A3':
        A3_image_paths.append(image)
        A3_counts.append(count)
    else:
        A4_image_paths.append(image)
        A4_counts.append(count)

print("Data points in folder A1:", len(A1_image_paths))
print("Data points in folder A2:", len(A2_image_paths))
print("Data points in folder A3:", len(A3_image_paths))
print("Data points in folder A4:", len(A4_image_paths))

Data points in folder A1: 128
Data points in folder A2: 31
Data points in folder A3: 27
Data points in folder A4: 624


## Helper functions

In [ ]:
def get_images(image_path, size = [256, 256]):
    # Read the image path
    image = tf.io.decode_png(tf.io.read_file(image_path), channels = 3)
    image = tf.image.convert_image_dtype(image, tf.float64)
    # Resizing
    #image = tf.image.resize(image, [448, 448])  # Based on the values from the paper
    image = tf.image.resize(image, size)
    return image

def get_MSE(counts, predictions):
    return mean_squared_error(counts, predictions)

def get_avg_MSE(mse_A1, mse_A2, mse_A3, mse_A4):
    return (mse_A1 + mse_A2 + mse_A3 + mse_A4) / 4 

## Prepare the data for the models

In [ ]:
A1_images = np.zeros(shape=(128, 256, 256, 3))
for i, (image, count) in enumerate(zip(A1_image_paths, A1_counts)):
     image = get_images(image, size = [256,256])
     A1_images[i] = image
A1_counts = np.array(A1_counts)

A2_images = np.zeros(shape=(31, 256, 256, 3))
for i, (image, count) in enumerate(zip(A2_image_paths, A2_counts)):
     image = get_images(image, size = [256,256])
     A2_images[i] = image
A2_counts = np.array(A2_counts)    

A3_images = np.zeros(shape=(27, 256, 256, 3))
for i, (image, count) in enumerate(zip(A3_image_paths, A3_counts)):
     image = get_images(image, size = [256,256])
     A3_images[i] = image
A3_counts = np.array(A3_counts)

A4_images = np.zeros(shape=(624, 256, 256, 3))
for i, (image, count) in enumerate(zip(A4_image_paths, A4_counts)):
     image = get_images(image, size = [256,256])
     A4_images[i] = image
A4_counts = np.array(A4_counts)

## Unquantized model results:

In [ ]:
# Load the model
model_path = "Transfer_learning_models/DenseNet121/"
model = tf.keras.models.load_model(model_path)

A1_preds = model.predict(A1_images)
A1_preds = A1_preds.reshape(A1_preds.shape[0])
A2_preds = model.predict(A2_images)
A2_preds = A2_preds.reshape(A2_preds.shape[0])
A3_preds = model.predict(A3_images)
A3_preds = A3_preds.reshape(A3_preds.shape[0])
A4_preds = model.predict(A4_images)
A4_preds = A4_preds.reshape(A4_preds.shape[0])

for i in range(len(A1_preds)):
    A1_preds[i] = round(A1_preds[i])
A1_preds = A1_preds.astype('int32')

for i in range(len(A2_preds)):
    A2_preds[i] = round(A2_preds[i])
A2_preds = A2_preds.astype('int32')

for i in range(len(A3_preds)):
    A3_preds[i] = round(A3_preds[i])
A3_preds = A3_preds.astype('int32')

for i in range(len(A4_preds)):
    A4_preds[i] = round(A4_preds[i])
A4_preds = A4_preds.astype('int32')


mse_1 = get_MSE(A1_preds, A1_counts)

mse_2 = get_MSE(A2_preds, A2_counts)

mse_3 = get_MSE(A3_preds, A3_counts)

mse_4 = get_MSE(A4_preds, A4_counts)

avg_MSE = get_avg_MSE(mse_1, mse_2, mse_3, mse_4)

print("MSE for A1: ", mse_1)
print("MSE for A2: ", mse_2)
print("MSE for A3: ", mse_3)
print("MSE for A4: ", mse_4)
print("Average MSE: ", avg_MSE)

MSE for A1:  0.7734375
MSE for A2:  2.870967741935484
MSE for A3:  2.0
MSE for A4:  1.2307692307692308
Average MSE:  1.7187936181761787


## Quantized model results:

In [ ]:
# Load the model
model_path = "Tflite_models/optimized_densenet_model.tflite"
interpreter_quant = tf.lite.Interpreter(model_path=model_path)
interpreter_quant.allocate_tensors()
input_index = interpreter_quant.get_input_details()[0]["index"]
output_index = interpreter_quant.get_output_details()[0]["index"]

A1_preds = []
for image in tqdm(A1_images):
    image = image.reshape(1, 256, 256, 3)
    interpreter_quant.set_tensor(input_index, np.array(image, dtype=np.float32))
    interpreter_quant.invoke()
    prediction = interpreter_quant.get_tensor(output_index)[0][0]
    A1_preds.append(round(prediction))

A2_preds = []
for image in tqdm(A2_images):
    image = image.reshape(1, 256, 256, 3)
    interpreter_quant.set_tensor(input_index, np.array(image, dtype=np.float32))
    interpreter_quant.invoke()
    prediction = interpreter_quant.get_tensor(output_index)[0][0]
    A2_preds.append(round(prediction))

A3_preds = []
for image in tqdm(A3_images):
    image = image.reshape(1, 256, 256, 3)
    interpreter_quant.set_tensor(input_index, np.array(image, dtype=np.float32))
    interpreter_quant.invoke()
    prediction = interpreter_quant.get_tensor(output_index)[0][0]
    A3_preds.append(round(prediction))



KeyboardInterrupt: ignored

In [ ]:
A4_preds = []
for image in tqdm(A4_images):
    image = image.reshape(1, 256, 256, 3)
    interpreter_quant.set_tensor(input_index, np.array(image, dtype=np.float32))
    interpreter_quant.invoke()
    prediction = interpreter_quant.get_tensor(output_index)[0][0]
    A4_preds.append(round(prediction))

mse_1 = get_MSE(A1_preds, A1_counts)

mse_2 = get_MSE(A2_preds, A2_counts)

mse_3 = get_MSE(A3_preds, A3_counts)

mse_4 = get_MSE(A4_preds, A4_counts)

avg_MSE = get_avg_MSE(mse_1, mse_2, mse_3, mse_4)

print("MSE for A1: ", mse_1)
print("MSE for A2: ", mse_2)
print("MSE for A3: ", mse_3)
print("MSE for A4: ", mse_4)
print("Average MSE: ", avg_MSE)


MSE for A1:  0.859375
MSE for A2:  2.6774193548387095
MSE for A3:  2.1481481481481484
MSE for A4:  1.2163461538461537
Average MSE:  1.725322164208253


## Results:

In [ ]:
from prettytable import PrettyTable

x = PrettyTable(['Model_type', 'MSE on A1', 'MSE on A2', 'MSE on A3', 'MSE on A4', 'Average MSE'])
x.add_row(['Unquantized_model', 0.7734, 2.8709, 2.0000, 1.2307, 1.7187])
x.add_row(['Quantized_model', 0.8593, 2.6774, 2.1481, 1.2163, 1.7253])

print(x)

+-------------------+-----------+-----------+-----------+-----------+-------------+
|     Model_type    | MSE on A1 | MSE on A2 | MSE on A3 | MSE on A4 | Average MSE |
+-------------------+-----------+-----------+-----------+-----------+-------------+
| Unquantized_model |   0.7734  |   2.8709  |    2.0    |   1.2307  |    1.7187   |
|  Quantized_model  |   0.8593  |   2.6774  |   2.1481  |   1.2163  |    1.7253   |
+-------------------+-----------+-----------+-----------+-----------+-------------+
